In [ ]:
import numpy as np
import pandas as pd
from sklearn import naive_bayes
import nltk
import re

In [ ]:
# load data
data = pd.read_csv('posts/shuffled_posts.csv', encoding = 'latin1')
relevant_ix = []
for index, row in data.iterrows():
    if row['is_relevant'] == 1:
        relevant_ix.append(index)
data_annotated = data.ix[relevant_ix]

In [ ]:
# preprocessing
text = []
stemmer = nltk.stem.porter.PorterStemmer()
for index, row in data_annotated.iterrows():
    full_text = row['status_message'].lower()
    tk_text = nltk.word_tokenize(full_text)
    stemmed = [stemmer.stem(token) for token in tk_text]
    text.append(stemmed)

In [ ]:
# construct vocabulary with ttf
k = 10
ttf = {}
for words in text:
    for word in words:
        if word not in ttf:
            ttf[word] = 1
        else:
            ttf[word] += 1
vocab = [word for word in ttf.keys() if ttf[word] >= k and not re.search('\d', word)] # high word count and not date

In [ ]:
# create feature vectors
X = []
for row in text:
    X_row = []
    for voc in vocab:
        if voc in row:
            X_row.append(1)
        else:
            X_row.append(0)
    X.append(X_row)
X = np.matrix(X)
#X_relevant = X.ix[X['is_relevant'] == 1]
y_driver = np.array(data_annotated['is_driver'])
y_roundtrip = np.array(data_annotated['is_roundtrip'])
#y_relevant = data_annotated['is_relevant']

In [ ]:
# machine learning!
clf = skl.naive_bayes.GaussianNB()
clf.fit(X, y_driver)
pred = clf.predict(X)

In [ ]:
# to do: model selection, evaluation, etc.
count = 0
for i in range(len(y_driver)):
    if y_driver[i] == pred[i]:
        count += 1
count